In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
def prediction_function(csv_path='./test_x.csv'):
    
    pca = pickle.load(open('./models/pca.pkl', 'rb'))
    KNN_uni = pickle.load(open('./models/knn_uni.pkl', 'rb'))
    KNN_dist = pickle.load(open('./models/knn_dist.pkl', 'rb'))
    SVC_rbf = pickle.load(open('./models/svc_rbf.pkl', 'rb'))
    RFC = pickle.load(open('./models/rfc.pkl', 'rb'))
    XGB = pickle.load(open('./models/xgb.pkl', 'rb'))
    
    X_test = pd.read_csv(csv_path).drop(['ID_Test'], axis=1)
    X_test = pd.get_dummies(X_test, columns=['Loan type', 'Occupation type'])
    X_test_pca = pca.transform(X_test)
    
    y_test_knn_dist = KNN_dist.predict(X_test_pca)
    y_test_knn_uni = KNN_uni.predict(X_test_pca)
    y_test_svc_rbf = SVC_rbf.predict(X_test_pca)
    y_test_rfc = RFC.predict(X_test_pca)
    y_test_xgb = XGB.predict(X_test_pca)
    
    y_test = 0.85*y_test_knn_uni + 0.85*y_test_knn_dist +\
             0.89*y_test_svc_rbf + 0.88*y_test_rfc + 0.88*y_test_xgb
           
    y_test = (y_test>=1.5)
    
    return y_test

In [3]:
y_test_pred = prediction_function()

ids = np.linspace(1, y_test_pred.shape[0], y_test_pred.shape[0])
pred_array = np.concatenate((ids.reshape(-1, 1), y_test_pred.reshape(-1, 1)), axis=1)
pred_df = pd.DataFrame(pred_array, columns=['ID', 'Label'], dtype=int)
pred_df.to_csv('./pred_y.csv')

display(pred_df)

,ID,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
19995,19996,1
19996,19997,1
19997,19998,1
19998,19999,1
